In [1]:
!nvidia-smi

Wed Dec  9 14:16:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
os.chdir('/content/drive/My Drive/GitHub/mitochondria_segmentation/src')
print(os.getcwd())
#print(os.listdir())

/content/drive/My Drive/GitHub/mitochondria_segmentation/src


In [ ]:
!python train.py

Data loaded.
device: cuda:0
train.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
train.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  lables = torch.tensor(lables, dtype=torch.long)
[1, loss: 0.255239]
2.0 minutes per epoch.
[2, loss: 0.167220]
2.0 minutes per epoch.
[3, loss: 0.143845]
2.0 minutes per epoch.
[4, loss: 0.126218]
2.0 minutes per epoch.
[5, loss: 0.113263]
2.0 minutes per epoch.
[6, loss: 0.103518]
2.0 minutes per epoch.
[7, loss: 0.096109]
2.0 minutes per epoch.
[8, loss: 0.089331]
2.0 minutes per epoch.
[9, loss: 0.083198]
2.0 minutes per epoch.


In [ ]:
!python predict.py

In [ ]:
!python messure.py

{'TP': 582689, 'TN': 11065829, 'FP': 19985, 'FN': 127977, 'acc': 0.9874571058485242, 'sn': 0.9668394521748059, 'sp': 0.9885671593736749, 'rc': 0.8199196246900783, 'f1': 0.8873391500660668, 'jac': 0.7974929206967474, 'dice': 0.8873391505626869}


In [4]:
!cat train.py

# -*- coding: utf-8 -*-
"""
Created on Fri Nov 13 22:23:58 2020

@author: zll
"""


import torch
from Mito_Dataset import Mito_Dataset
from torch.utils.data import DataLoader
from model import U_Net
from time import time


# ******** Hyper Parameters ********
# epoches till stop
EPOCHES = 1000
# if there is pretrained parameters
PRETRAIN = False
# epoches trained
pre_epoch = 0
# path to save parameter file
save_path = './param_128/'
# pretrained model parameter
pretrained = save_path + 'param.pkl'
# batch size
batch_size = 128
# **********************************


dataset = Mito_Dataset()
data_loader = DataLoader(dataset, batch_size=batch_size)
print('Data loaded.')

# use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

# new and init model
model = U_Net()
if PRETRAIN:
    model.load_state_dict(torch.load(pretrained, map_location=device))
model.to(device)  # copy model to GPU

# loss function
criterion = torch.nn.Cross